In [1]:
from memory import MemoryBuffer
from policies import PPOTrainer
import tensorflow as tf
from tensorflow.layers import dense, conv2d, max_pooling2d, flatten
import numpy as np
import time
import gym

In [2]:
# Pacman network

env = gym.make('MsPacman-v0')

obs = tf.placeholder(tf.float32, shape=[None]+list(env.observation_space.shape))
pool1 = max_pooling2d(obs, 4, 4)
conv1 = conv2d(pool1, 16, 5, activation=tf.tanh)
pool2 = max_pooling2d(conv1, 2, 2)
conv2 = conv2d(pool2, 16, 3, activation=tf.tanh)
flattened = flatten(conv2)
act_probs = dense(flattened, env.action_space.n)
softmax_probs = tf.nn.softmax(act_probs)

v_conv1 = conv2d(pool1, 16, 5, activation=tf.tanh)
v_pool2 = max_pooling2d(v_conv1, 2, 2)
v_conv2 = conv2d(v_pool2, 16, 3, activation=tf.tanh)
v_flattened = flatten(v_conv2)
value = dense(v_flattened, 1)

network = PPOTrainer(obs, softmax_probs, value, act_type='d')

/home/ejmejm/anaconda3/lib/python3.6/site-packages/gym/envs/registration.py:14: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
  result = entry_point.load(False)


In [3]:
n_episodes = 1000000
max_steps = 200
update_freq = 4
print_freq = 1

mb = MemoryBuffer()

In [4]:
all_rewards = []

for episode in range(n_episodes):
    ep_reward = 0
    
    mb.start_rollout()
    obs = env.reset()
    for step in range(max_steps):
        obs = obs.squeeze()
        obs = obs / 255.
        act = network.gen_act(obs)
        
        obs_next, rew, d, _ = env.step(act)
        ep_reward += rew
        
        if False:
            env.render()
            time.sleep(0.02)
        
        mb.record(obs, act, rew)
        obs = obs_next
        
        if d:
            break
            
    all_rewards.append(ep_reward)
            
    if episode % update_freq == 0 and episode != 0:
        network.train(mb.to_data())
        
        if episode % (update_freq * print_freq) == 0:
            print(f'Update #{episode // update_freq}, Reward: {np.mean(all_rewards[-update_freq*print_freq:])}')

InternalError: cudnn PoolForward launch failed
	 [[node max_pooling2d/MaxPool (defined at <ipython-input-2-3ed18bcc141a>:6)  = MaxPool[T=DT_FLOAT, data_format="NCHW", ksize=[1, 1, 4, 4], padding="VALID", strides=[1, 1, 4, 4], _device="/job:localhost/replica:0/task:0/device:GPU:0"](max_pooling2d/MaxPool-0-TransposeNHWCToNCHW-LayoutOptimizer)]]

Caused by op 'max_pooling2d/MaxPool', defined at:
  File "/home/ejmejm/anaconda3/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/ejmejm/anaconda3/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/ejmejm/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/ejmejm/anaconda3/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/ejmejm/anaconda3/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 505, in start
    self